In [1]:
import torch

In [2]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars =  sorted(set(text))

vocabulary_size = len(chars)

print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [3]:

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 66, 77, 69, 62, 26,  1, 32, 72, 75, 72, 77, 65, 82,  1, 58, 71,
        61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,  1, 66, 71,  1, 43, 83,
         0,  0, 29, 78, 77, 65, 72, 75, 26,  1, 40, 14,  1, 34, 75, 58, 71, 68,
         1, 30, 58, 78, 70,  0,  0, 37, 69, 69, 78, 76, 77, 75, 58, 77, 72, 75,
        26,  1, 38, 72, 65, 71,  1, 46, 14,  1, 42, 62, 66, 69, 69,  0,  0, 46,
        62, 69, 62, 58, 76, 62,  1, 61, 58, 77])


In [4]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

class BigramLanguageModel(torch.nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = torch.nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = torch.nn.functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # (B,C)
            probs = torch.nn.functional.softmax(logits, dim=-1)  # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx
model = BigramLanguageModel(vocabulary_size)
model.apply(weights_init)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
batch_size = 32
block_size = 8
def get_batch(split):
    data_split = data if split == 'train' else data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])
    return x, y
for step in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(f"step {step}: loss {loss.item()}")
context = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

step 0: loss 4.917428970336914
step 1000: loss 3.8352036476135254
step 2000: loss 3.071303606033325
step 3000: loss 2.7841734886169434
step 4000: loss 2.559861183166504
step 5000: loss 2.6741554737091064
step 6000: loss 2.564612865447998
step 7000: loss 2.5171751976013184
step 8000: loss 2.4360389709472656
step 9000: loss 2.553037405014038



Wiza8llll lothelseht, wighison_whem agan " bulothend nd l st T."ofangatotingrke acks mad then ted ainenoonghed pe caryoornd Onobeeto thaninss as.
d tr mer oul imid er the y Pre bseng.
t ghre.5he "onopeaiend tre rushes h these. Yedie faid Thomindenthered al al me g, sa st adek,64s toowhase o tag LL2_Whid t fureBus troule,. thatly orlane?"I'musck, brt ld g,"
a way hasth(Ju,"WI ony, aton d  ary
ad Resa om s wl!mlly. ileridce t bu. t hek an thaly6-ftroghand thy fon ifout Gucond bze Sosph too, aly 
